In [ ]:
!pip install ktoolbox
!mkdir -p /kaggle/working/ktoolbox && \
cd /kaggle/working/ktoolbox && \
ktoolbox sync-creator https://kemono.su/fanbox/user/49494721


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=efd1032dbdfb1cc2f7d82da1472a9c835641d3c28fca2e9f7a8f730c3ec7263d
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
  Attempting uninstall: aiofiles
    Found existing installation: aiofiles 22.1.0
    Uninstalling aiofiles-22.1.0:
      Successfully uninstalled aiofiles-22.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ypy-websocket 0.8.4 requires aiofiles<23,>=22.1.0, but you have aiofiles 23.2.1 which is incompatible.
2025

In [ ]:
import os
import shutil
from PIL import Image

src_dir = '/kaggle/working/ktoolbox'  # 原始目录
dst_dir = '/kaggle/working/work'  # 目标目录

os.makedirs(dst_dir, exist_ok=True)

for root, _, files in os.walk(src_dir):
    for file in files:
        file_path = os.path.join(root, file)
        try:
            with Image.open(file_path) as img:
                img.verify()
            shutil.move(file_path, os.path.join(dst_dir, file))
        except:
            continue


In [ ]:
import os
from PIL import Image, ImageChops

def is_color_image(img, threshold=10):
    if img.mode != 'RGB':
        img = img.convert('RGB')

    r, g, b = img.split()
    diff_rg = ImageChops.difference(r, g)
    diff_rb = ImageChops.difference(r, b)
    diff_gb = ImageChops.difference(g, b)

    def mean_diff(diff_img):
        hist = diff_img.histogram()
        pixels = sum(hist)
        total = sum(i * hist[i] for i in range(256))
        return total / pixels if pixels else 0

    mean_rg = mean_diff(diff_rg)
    mean_rb = mean_diff(diff_rb)
    mean_gb = mean_diff(diff_gb)

    if mean_rg > threshold or mean_rb > threshold or mean_gb > threshold:
        return True
    else:
        return False

def clean_work_folder():
    work_dir = '/kaggle/working/work'
    for file in os.listdir(work_dir):
        path = os.path.join(work_dir, file)
        if not file.lower().endswith(('.jpg', '.jpeg', '.png')):
            os.remove(path)
            continue
        try:
            with Image.open(path) as img:
                if img.width < 256 or img.height < 256 or not is_color_image(img):
                    img.close()
                    os.remove(path)
        except Exception as e:
            print(f"处理失败: {path}，错误：{e}")
            if os.path.exists(path):
                os.remove(path)

if __name__ == '__main__':
    clean_work_folder()


In [ ]:
import shutil
shutil.rmtree("/kaggle/working/ktoolbox")
shutil.make_archive('/kaggle/working/result', 'zip', '/kaggle/working/work')
